In [2]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 19.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 6.8 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=1c608fd4c9eec2f5f591d979d18462a77da1ef59cfe86c4dcca11ef3bd80b2cf
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

import pickle as pkl

In [5]:
wandb_project_name = "crowd-forecasting"

scaler_save_path = '/content/drive/Othercomputers/My Laptop/github-repositories/snfc-crowd-forecasting/models/mlp/scaler.pkl'
scaler_reload_path = '/content/drive/Othercomputers/My Laptop/github-repositories/snfc-crowd-forecasting/models/mlp/scaler.pkl'

In [4]:
import wandb
from wandb.keras import WandbCallback

wandb.init(project=wandb_project_name)

#wandb
wandb_callback = WandbCallback(monitor='val_loss', mode='min', log_gradients=False,save_model=False)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
# Read full dataset (one hot encoding)
train = pd.read_csv('/content/drive/Othercomputers/My Laptop/github-repositories/snfc-crowd-forecasting/data/train-ohe.csv')
test = pd.read_csv('/content/drive/Othercomputers/My Laptop/github-repositories/snfc-crowd-forecasting/data/test-ohe.csv')

In [7]:
# Re-train on full data, to upload to dashboard competition benchmark
train_on_full = False

In [8]:
if(train_on_full == True):
    data = [train, test]
    train = pd.concat(data)

    train.reset_index(drop=True, inplace=True)

    test = train

In [9]:
train.columns

Index(['p1q0', 'p2q0', 'p3q0', 'p0q1', 'p0q2', 'p0q3', 'p0q0', 'month_April',
       'month_February', 'month_January',
       ...
       'station_BA', 'station_BB', 'station_BC', 'station_BD', 'station_BE',
       'station_BF', 'station_BG', 'station_BH', 'station_BI', 'station_BJ'],
      dtype='object', length=112)

In [10]:
# Splitting of features

x_train = train.loc[:, train.columns != 'p0q0']
y_train = train.p0q0

# Shuffle
x_train, y_train = shuffle(x_train, y_train, random_state=0)

x_test = test.loc[:, test.columns != 'p0q0']
y_test = test.p0q0

print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)


(21165, 111) (21165,)
(9952, 111) (9952,)


In [11]:
# Scaling

def scale_full_dataframe(dataframe, scaler_reload_path):

    dataframe_scaled = dataframe[["p1q0","p2q0","p3q0","p0q1","p0q2","p0q3"]]
    dataframe = dataframe.drop(columns=["p1q0","p2q0","p3q0","p0q1","p0q2","p0q3"], axis=1) # Keep only meta-data features

    if scaler_reload_path is not None: # if this is test set
        scaler = pd.read_pickle(scaler_reload_path)
    else:
        scaler = StandardScaler()

    dataframe_scaled = pd.DataFrame(scaler.fit_transform(dataframe_scaled), columns=["p1q0","p2q0","p3q0","p0q1","p0q2","p0q3"])

    dataframe = pd.concat([dataframe, dataframe_scaled], axis=1)
    
    if scaler_reload_path is None:  # if this is training set save scaler to file
        # Save scaler to file
        
        with open(scaler_save_path, "wb") as outfile:
            pkl.dump(scaler, outfile)

    return dataframe

In [12]:
x_train.head(2)

,p1q0,p2q0,p3q0,p0q1,p0q2,p0q3,month_April,month_February,month_January,month_March,...,station_BA,station_BB,station_BC,station_BD,station_BE,station_BF,station_BG,station_BH,station_BI,station_BJ
3148,0.085,0.287,0.383,0.000,0.000,0.000,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
16357,0.088,0.146,0.164,0.036,0.041,0.075,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# All Features
x_train_sub = x_train
x_test_sub = x_test


x_train_sub = scale_full_dataframe(x_train_sub, scaler_reload_path = None)  
x_test_sub = scale_full_dataframe(x_test_sub, scaler_reload_path = scaler_reload_path)  


In [14]:
x_train_sub.head(5)

,month_April,month_February,month_January,month_March,month_May,day_Friday,day_Monday,day_Thursday,day_Tuesday,day_Wednesday,...,station_BG,station_BH,station_BI,station_BJ,p1q0,p2q0,p3q0,p0q1,p0q2,p0q3
0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,-0.792793,0.414876,0.671072,-1.304219,-1.109167,-0.938706
1,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,-0.775468,-0.465113,-0.499507,-1.056124,-0.855648,-0.461404
2,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,-1.007630,-1.050524,-0.563649,-1.304219,-1.109167,-0.938706
3,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,-0.226826,-0.153060,-0.360535,0.074084,-0.070357,-0.111383
4,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0.616350,-0.121855,-0.355189,0.487575,0.714934,0.575932


In [15]:
x_train_sub.shape

(21165, 111)

In [16]:
import tensorflow as tf

input_layer = tf.keras.Input(shape=(111,)) 

#x = tf.keras.layers.Dense(2048, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01))(input_layer)
#x = tf.keras.layers.Dense(2048, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.02))(x)
#x = tf.keras.layers.Dense(2048, activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.03))(x)
#x = tf.keras.layers.Dense(2048, activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.03))(x)

x = tf.keras.layers.Dense(2048, activation='relu')(input_layer)
x = tf.keras.layers.Dense(2048, activation='relu')(x)
x = tf.keras.layers.Dense(2048, activation='relu')(x)
x = tf.keras.layers.Dense(2048, activation='relu')(x)


# Sigmoid because we want to squash the values, the crowd factor is always between 0 and 1
output_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(x) 

model = tf.keras.Model(input_layer, output_layer)

model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['mse', 'mae'])


# early stopping
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta = 0.000001, patience=10, mode='min', restore_best_weights=True) 

# callbacks
callbacks_list = [wandb_callback,early_stopping_callback]

In [17]:
# Model training
from sklearn.utils import shuffle
model.fit(
    x_train_sub, y_train, 
    validation_split=0.2,
    batch_size = 32, 
    epochs=200,  #1000
    shuffle = True,
    callbacks=callbacks_list,
)

Epoch 1/200
530/530 [==============================] - 10s 7ms/step - loss: 0.0033 - mse: 0.0033 - mae: 0.0395 - val_loss: 0.0022 - val_mse: 0.0022 - val_mae: 0.0343
Epoch 2/200
530/530 [==============================] - 3s 6ms/step - loss: 0.0017 - mse: 0.0017 - mae: 0.0300 - val_loss: 0.0019 - val_mse: 0.0019 - val_mae: 0.0338
Epoch 3/200
530/530 [==============================] - 5s 9ms/step - loss: 0.0013 - mse: 0.0013 - mae: 0.0261 - val_loss: 0.0017 - val_mse: 0.0017 - val_mae: 0.0294
Epoch 4/200
530/530 [==============================] - 3s 6ms/step - loss: 0.0011 - mse: 0.0011 - mae: 0.0237 - val_loss: 0.0014 - val_mse: 0.0014 - val_mae: 0.0268
Epoch 5/200
530/530 [==============================] - 3s 6ms/step - loss: 8.9959e-04 - mse: 8.9959e-04 - mae: 0.0219 - val_loss: 0.0014 - val_mse: 0.0014 - val_mae: 0.0277
Epoch 6/200
530/530 [==============================] - 3s 6ms/step - loss: 7.4444e-04 - mse: 7.4444e-04 - mae: 0.0201 - val_loss: 0.0016 - val_mse: 0.0016 - val_mae: 

In [18]:
y_pred = model.predict(x_train_sub)
mae_train = mean_absolute_error(y_pred, y_train)

y_pred = model.predict(x_test_sub)
mae_test = mean_absolute_error(y_pred, y_test)

print("mae train: ", mae_train)
print("mae test: ", mae_test)

311/311 [==============================] - 1s 2ms/step
mae train:  0.007608360111777829
mae test:  0.04176852608820233


In [19]:
model.save('/content/drive/Othercomputers/My Laptop/github-repositories/snfc-crowd-forecasting/models/mlp/')